In [ ]:
import pandas as pd
from pickle import dump
from pickle import load
pd.set_option("max_columns", None)
pd.set_option('max_rows', None)

df = pd.read_csv('poverty.csv')

df.head()

df.columns = df.iloc[0]

df.head()

df.drop(df.index[0], inplace = True)

df.info()

df.head()

Remove columns that contain the word 'Bound'. These are not needed.

df = df.loc[:, ~df.columns.str.contains('Bound')]

Remove columns that are percentages.

df = df.loc[:, ~df.columns.str.contains('Percent')]

Locate DC and change County FIPS so it isn't dropped during the next step.

df.loc[(df['County FIPS'] == '000') & (df['Name'] == 'District of Columbia')]

df.iloc[327:330, :]

Nevermind. DC is in there alone and summed (as if it was a state). Just drop all County FIPS 000.

Drop all rows with county FIPS 000. These are just states summed. We don't need them. Might be useful to look at for any missing information later though.

drop = df.loc[(df['County FIPS'] == '000')]

df = df.drop(drop.index, axis = 0)

Check on the Hawaii counties. May need to drop Kalawao (no election results. Very low pop) Change states to full words. Look for Puerto Rico. Strip county, borough, etc. 

df.loc[(df['Postal'] == 'HI')]

Kalawao = (pd.DataFrame(df.loc[562]))
df = df.drop(Kalawao)

df.head()

us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
}

df.Postal = df.Postal.map(us_state_abbrev)

df.head()

df = df.drop(df.iloc[:, [0, 1, 6, 8]], axis = 1)

df.head()

df['Name'] = df['Name'].str.replace('(Borough|County|Area|Census|Municipality|and|City|Municipali)', '')

Chugach and Copper River are already combined into Valdez-Cordova so no need to change that. Renaming Wade Hampton to Kusilvak to match results.

df.loc[df['Postal'] == 'Alaska']

df.loc[96]['Name'] = 'Kusilvak'

Remove punctuation.

def remove_punctuation(x):
    try:
        x = x.str.replace('[^\w\s]','')
    except:
        pass
    return x
df = df.apply(remove_punctuation)

df.head()

df.columns = ['state', 'county', 'poverty_total', 'poverty_under_18', 'median_household_income']

df = df.sort_values(by = ['state', 'county']).reset_index(drop = True)

df.info()

df[df.columns[2:]] = df[df.columns[2:]].astype(float)

df.info()

df.head()

race = load(open('results_race_with_alaska.pkl', 'rb'))

df.loc[~df['county'].isin(race['County'])]

